<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Sistema de obtención de información con NLTK utilizando un corpus de wikipedia


In [1]:
import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario

###solo al primer uso
#nltk.download("punkt")
#nltk.download("wordnet")
#nltk.download('omw-1.4')

### Datos
Se consumirán los datos del artículo de wikipedia sobre el deporte "Tennis" en inglés.

In [28]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Colombia')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [29]:
# Demos un vistazo
article_text

'\n\ncolombia, oficialmente república de colombia, es un país soberano situado en la región noroccidental de américa del sur. se constituye en un estado unitario, social y democrático de derecho cuya forma de gobierno es presidencialista con dos cámaras legislativas. su capital y ciudad más poblada es bogotá.[12]\u200b es una república organizada políticamente en treinta y dos departamentos descentralizados y el distrito capital de bogotá,[13]\u200b sede del gobierno nacional.\nincluyendo la isla de malpelo, el cayo roncador y el banco serrana, el país abarca una superficie de 1 141 748 km²,[3]\u200b por lo que es el vigesimosexto país más grande del mundo y el séptimo más grande de américa. reclama como mar territorial el área hasta las 12 millas náuticas de distancia,[4]\u200b manteniendo un diferendo limítrofe al respecto con venezuela y nicaragua.[14]\u200b[15]\u200b\nlimita al oriente con venezuela y brasil, al sur con perú y ecuador y al occidente con panamá; en cuanto a límites 

In [30]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 108895


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [31]:
import unicodedata

#retirar acentos
text = unicodedata.normalize('NFKD', article_text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [33]:
# Repaso de regex:
# https://docs.python.org/3/library/re.html

# Para practicar regex:
# https://regex101.com/

# el inicio con 'r' antes de cada string indica que se interprete como raw string
# '\n' es interpretado por Python como salto de linea
# r'\n' es interpretado por Python como el string formado por dos caracteres: 
#  backslash y n

# substituir con regex con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', text) # substituir los números entre corchetes
# (notar que los corchetes son interpretados literalmente por los backlsash)
text = re.sub(r'\s+', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación
text = re.sub(r'\\u200b', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación

# probar en regex101 con los patrones anteriores:
# 'Hola [1], [], [ estoy bien   [123]. [12sss]. OK!   .'

In [34]:
# Demos un vistazo
text

' colombia, oficialmente republica de colombia, es un pais soberano situado en la region noroccidental de america del sur. se constituye en un estado unitario, social y democratico de derecho cuya forma de gobierno es presidencialista con dos camaras legislativas. su capital y ciudad mas poblada es bogota. es una republica organizada politicamente en treinta y dos departamentos descentralizados y el distrito capital de bogota, sede del gobierno nacional. incluyendo la isla de malpelo, el cayo roncador y el banco serrana, el pais abarca una superficie de 1 141 748 km2, por lo que es el vigesimosexto pais mas grande del mundo y el septimo mas grande de america. reclama como mar territorial el area hasta las 12 millas nauticas de distancia, manteniendo un diferendo limitrofe al respecto con venezuela y nicaragua. limita al oriente con venezuela y brasil, al sur con peru y ecuador y al occidente con panama; en cuanto a limites maritimos, colinda con panama, costa rica, nicaragua, honduras,

In [35]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 106502


### 3 - Dividir el texto en sentencias y en palabras

In [36]:
corpus = nltk.sent_tokenize(text) # divide en oraciones
words = nltk.word_tokenize(text) # divide en términos

In [38]:
len(corpus)

575

In [39]:
# Demos un vistazo
corpus[:-20]

[' colombia, oficialmente republica de colombia, es un pais soberano situado en la region noroccidental de america del sur.',
 'se constituye en un estado unitario, social y democratico de derecho cuya forma de gobierno es presidencialista con dos camaras legislativas.',
 'su capital y ciudad mas poblada es bogota.',
 'es una republica organizada politicamente en treinta y dos departamentos descentralizados y el distrito capital de bogota, sede del gobierno nacional.',
 'incluyendo la isla de malpelo, el cayo roncador y el banco serrana, el pais abarca una superficie de 1 141 748 km2, por lo que es el vigesimosexto pais mas grande del mundo y el septimo mas grande de america.',
 'reclama como mar territorial el area hasta las 12 millas nauticas de distancia, manteniendo un diferendo limitrofe al respecto con venezuela y nicaragua.',
 'limita al oriente con venezuela y brasil, al sur con peru y ecuador y al occidente con panama; en cuanto a limites maritimos, colinda con panama, costa r

In [40]:
# Demos un vistazo
words[:20]

['colombia',
 ',',
 'oficialmente',
 'republica',
 'de',
 'colombia',
 ',',
 'es',
 'un',
 'pais',
 'soberano',
 'situado',
 'en',
 'la',
 'region',
 'noroccidental',
 'de',
 'america',
 'del',
 'sur']

In [41]:
print("Vocabulario:", len(words))

Vocabulario: 19386


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [43]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# ord() nos da el código Unicode para un caracter dado
punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula (string.lower())
    # 2 - quitar los simbolos de puntuacion (string.translate())
    # 3 - realiza la tokenización (nltk.word_tokenize)
    # 4 - realiza la lematización (nuestra función perform_lemmatization)
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='spanish')

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias a ensayar:
- Grand slam
- tournaments
- nadal
- artificial intelligence

In [26]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
#!{sys.executable} -m pip install gradio --quiet

In [49]:
import gradio as gr

def bot_response(human_text):
    print("Q:", human_text)    
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

C:\Users\Jesus\AppData\Local\Temp\ipykernel_9392\1968786774.py:9: GradioDeprecationWarning: `layout` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Q: libertador


Traceback (most recent call last):
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\gradio\routes.py", line 439, in run_predict
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\gradio\blocks.py", line 1384, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\gradio\blocks.py", line 1089, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Je

Q: libertador


...

Q: libertador


Traceback (most recent call last):
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\gradio\routes.py", line 439, in run_predict
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\gradio\blocks.py", line 1384, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\gradio\blocks.py", line 1089, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Je

Q: libertador


Traceback (most recent call last):
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\gradio\routes.py", line 439, in run_predict
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\gradio\blocks.py", line 1384, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\gradio\blocks.py", line 1089, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jesus\Documents\CEIA\Procesamiento de lenguaje natural\env\Lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Je

Keyboard interruption in main thread... closing server.


### Alumno

- Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
- Sacar conclusiones de los resultados.

__IMPORTANTE__: Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.